In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def exp(x, a, b, c,d,e):
    global wavelengthG
    x = np.array(x)
    expR = np.exp(-b/x)
    return a*(1-expR-c*expR**2-d*expR**3-e*expR**4)#-np.exp(-fThickness(wavelengthG)/x))#-np.exp(-c/x))

def expInv(y, *args):
    def diff(x, *args):
        
        yt = exp(x, *args)
        return (yt - y )**2
    res = minimize(diff, 78, args=(args), method='Nelder-Mead', tol=1e-6)
    return res.x[0]

@dataclass
class SimuFile:
    abslength : float
    wavelength : np.ndarray
    maxCounts : np.ndarray
    counts : np.ndarray
    
    def __post_init__(self):
        self.maxCounts = np.mean(self.maxCounts)
        self.QE = self.counts/self.maxCounts
        self.QE_err = np.sqrt(self.counts)/self.maxCounts
        
    def getQEatWavelength(self, wavelength):
        return self.QE[self.wavelength==wavelength][0], self.QE_err[self.wavelength==wavelength][0]
    
class AbsLengthFromQE:
    def __init__(self):
        self.absLengths = []
        self.dic = {}
        self.interpolationkind = "quadratic"
        
    def appendDataSet(self, file : SimuFile):
        self.absLengths.append(file.abslength)
        self.dic[file.abslength] = file
        
    def GetQEvsAbs(self, wavelength):
        QEs = []
        Qeerr = []
        for key in self.absLengths:
            qe, err = self.dic[key].getQEatWavelength(wavelength)
            QEs.append(qe)
            Qeerr.append(err)
        return np.array(QEs), np.array(Qeerr)
    
    def fit_exp(self, wavelength, y, error):
        global wavelengthG
        wavelengthG = wavelength
        popt,pcov = curve_fit(exp, np.array(self.absLengths), y,
                              sigma=error, p0 = [lossGlass(wavelength), 20,1,1,1], absolute_sigma=1, maxfev = 10000)
        return popt
        
    def matchQEdata(self,QE, wavelength):
        data = self.GetQEvsAbs(wavelength)
        popt = self.fit_exp(wavelength,*data)
        f = interp1d(data[0],  np.array(self.absLengths), kind =  self.interpolationkind, fill_value="extrapolate")
        
        global wavelengthG
        wavelengthG = wavelength
        return expInv(QE, *popt), popt#f(QE)
    
    def plotAbsVsQE(self, wavelength):
        plt.figure()
        data = self.GetQEvsAbs(wavelength)
        popt = self.fit_exp(wavelength,*data)
        print(popt)
        f = interp1d(np.array(self.absLengths), data[0],self.interpolationkind)
        plt.errorbar(self.absLengths, data[0], yerr = data[1], fmt = '.')
        
        x = np.logspace(0,4.2,100)
        plt.plot(x, exp(x, *popt), label = "fit")
        #plt.plot(x, f(x), label = "interpolation")
        plt.legend()
        plt.yscale("log")
        plt.xscale("log")
        plt.figure()
        plt.errorbar(self.absLengths, (exp(self.absLengths, *popt)-data[0])*100/data[0], yerr = data[1]/data[0], fmt = '.')
        plt.xscale("log")
        
absLengths = [10, 50, 70, 75, 80, 90, 100, 110, 120, 130, 140, 150, 200, 300,  600, 700, 800, 1000,3000, 10000, 11000]    
absLengths = [1,5, 10, 20, 30, 50, 60,  70, 75, 78, 80, 90, 100, 110, 120, 150, 200, 300,  500, 750, 1000, 10000]       
clc = AbsLengthFromQE()
for absL in absLengths: 
    try:
        clc.appendDataSet(
        SimuFile(absL,
                 *np.loadtxt(f"QE_transmission_simu_data/220815_correct_divergence_QE_erlangen/{absL}nm.txt", unpack=1, usecols=(2,11,12), skiprows=1)))
    except Exception as err:
        print(err)
wvl = 570

clc.plotAbsVsQE(wvl)
print(fQE(wvl))
print(len(absLengths))